In [ ]:
import web
import json
import sys
from time import time
import glob
import logging
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
sys.argv = ["prog", "0.0.0.0:80"]


def gen_clf():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')
    lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
    n_samples, h, w = lfw_people.images.shape
    print(n_samples)
    print(h)
    print(w)
    X = lfw_people.data
    n_features = X.shape[1]
    y = lfw_people.target
    target_names = lfw_people.target_names
    n_classes = target_names.shape[0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    #n_components = 150
    n_components = 5
    pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True).fit(X_train)
    eigenfaces = pca.components_.reshape((n_components, h, w))
    X_train_pca = pca.transform(X_train)
    #X_test_pca = pca.transform(X_test)
    param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
    clf = clf.fit(X_train_pca, y_train)
    #y_pred = clf.predict(X_test_pca)
    return clf, pca, target_names, X_test

clf, pca, target_names, X_test = gen_clf()

urls = (
    '/picfun/search', 'search',
    '/images/(.*)', 'search_result'
)
app = web.application(urls, globals())

class search_result:        
    def GET(self, tdir):
        web.header('Content-type','image/jpeg')
        fntmpl = 'C:/hackthon/data/lfw_funneled/%s/*' % tdir
        fns = glob.glob(fntmpl)
        fn = fns[np.random.randint(0, len(fns))]
        with open(fn, "rb") as fin:
            output = fin.read()
        return output
    
class search:        
    def POST(self):
        data_in = json.loads(web.data())
        print data_in["points"]
        points = data_in["points"]
        dpi = 250
        fig = plt.figure(frameon=False, dpi=dpi)
        fig.set_size_inches(250 / dpi, 250 / dpi)
        ax = plt.Axes(fig, [0., 0., 1.0, 1.0])
        ax.set_xlim([0, 250])
        ax.set_ylim([0, 250])
        ax.set_axis_off()
        fig.add_axes(ax)
        for (x, y) in points:
            ax.scatter(x=[x], y=[249-y], c='k', s=5)
        fig.canvas.draw()
        fig.savefig('C:/hackthon/data/figure.png')
        data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        image = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        slice_=(slice(70, 195), slice(78, 172))
        resize=0.4
        from scipy.misc import imresize
        face = np.asarray(image[slice_], dtype=np.float32)
        face /= 255.0  # scale uint8 coded colors to the [0.0, 1.0] floats
        if resize is not None:
            face = imresize(face, resize)
        face = face.mean(axis=2)
        img = face.reshape(1, -1)
        #img = X_test[[20],:]
        x_pca = pca.transform(img)
        print x_pca.sum()
        y_pred = clf.predict(x_pca)
        print y_pred[0]
        tname = target_names[y_pred[0]]
        pyDict = {'path': '/images/' + tname.replace(' ', '_'), 'name': tname}
        web.header('Content-Type', 'application/json')
        return json.dumps(pyDict)
    
class search_text:        
    def GET(self):
        output = 'my face'
        return output

if __name__ == "__main__":
    app.run()